In [4]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText


Cloning into 'fastText'...
remote: Enumerating objects: 3953, done.
remote: Counting objects: 100% (987/987), done.
remote: Compressing objects: 100% (155/155), done.
remote: Total 3953 (delta 881), reused 844 (delta 830), pack-reused 2966
Receiving objects: 100% (3953/3953), 8.26 MiB | 18.83 MiB/s, done.
Resolving deltas: 100% (2513/2513), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fastText: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199771 sha256=d5641d058a6bb073f459558880aeef3c71f1bc8fab6c347c630f18d74f6ab569
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fastText


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import fasttext
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Flatten,LSTM,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.text import Tokenizer

In [ ]:

!pip install huggingface_hub

from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-hi-vectors", filename="model.bin")
model = fasttext.load_model(model_path)
model.words

In [6]:
filename="/content/drive/MyDrive/iisc/reverse_dictionary/hindi_fasttext_model.bin"


In [ ]:
#model.save_model(filename)

In [7]:
loaded_model = fasttext.load_model(filename)

data load

In [8]:
!pip3 install pandas_ods_reader
from pandas_ods_reader import read_ods
newdf=read_ods("/content/drive/MyDrive/iisc/reverse_dictionary/hi_3500.ods",headers=False)
newdf.columns.values[0]='text'
newdf.columns.values[1]='label'
newdf.head()
newdf.label=pd.Categorical(newdf.label)
newdf['label']=newdf['label'].cat.codes
newdf = newdf.dropna()
newdf.head()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ezodf: filename=ezodf-0.3.2-py2.py3-none-any.whl size=49002 sha256=788ef6915162de5aa41916c2314a9a0aa203da6247b71e42c73660e4c8a6eda8
  Stored in directory: /root/.cache/pip/wheels/50/09/62/ea2c44e6ffdd067d6c8d3f557af0734a195bd252d6901f23ca
Successfully built ezodf


,text,label
0,लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...,0
1,गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...,0
2,ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है,0
3,यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...,0
4,अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...,0


In [9]:
label=newdf['label']
input=newdf['text']
Y = pd.get_dummies(newdf['label']).values
print(Y)
onehot=Y

[[1 0 0]
 [1 0 0]
 [1 0 0]
 ...
 [0 1 0]
 [0 1 0]
 [0 1 0]]


In [10]:
input

0       लोग वतन तक खा जाते हैं इसका इसे यकीन नहींमान ज...
1       गुमनाम है वतन पर मिटने वाले लोग आतन्कवादियों स...
2       ज़ंजीर बदली जा रही थी मैं समझा था रिहाई हो गयी है
3       यूपी में बड़े स्तर पर दंगे करवा सकती है बीजेपी...
4       अंग्रेजी नहीं आती है इसलिए हिन्दी ट्विट ज्यादा...
                              ...                        
9072    RT @aajtak: #Mojo बजरंग बली का सबसे विशालकाय र...
9073    RT @aajtak: कहां अचानक स्कूल में पड़ा मंत्री क...
9074    RT @aajtak: #MoJo सुनिए कुलभूषण की दर्दभरी आपब...
9075    RT @aajtak: लव जेहाद के नाम पर क्या उत्तर प्रद...
9076    कश्मीर में हमारे जवानों से ऐसा सलूक? वीडियो दे...
Name: text, Length: 9077, dtype: object

tokenizing and making vocab

In [28]:
tokenizer = Tokenizer(nb_words=27855)
tokenizer.fit_on_texts(input)
sequences = tokenizer.texts_to_sequences(input)

word_index = tokenizer.word_index
index_word=tokenizer.index_word
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=100)

#label = to_categorical(np.asarray(label))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', Y.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = Y[indices]
nb_validation_samples = int(0.2* data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = Y[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = Y[-nb_validation_samples:]

/usr/local/lib/python3.10/dist-packages/keras/preprocessing/text.py:246: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn(


Found 21535 unique tokens.
Shape of data tensor: (9077, 100)
Shape of label tensor: (9077, 3)


In [15]:
print(len(word_index))
print(index_word)

21535
{1: 'के', 2: 'में', 3: 'की', 4: 'है', 5: 'https', 6: 't', 7: 'co', 8: 'से', 9: 'को', 10: 'का', 11: 'rt', 12: 'पर', 13: 'और', 14: 'aajtak', 15: 'ने', 16: 'है।', 17: 'नहीं', 18: 'हैं', 19: 'भी', 20: 'इस', 21: 'कि', 22: 'लिए', 23: 'एक', 24: 'हो', 25: 'ही', 26: 'कर', 27: 'तो', 28: 'साथ', 29: 'दिल्ली', 30: 'हैं।', 31: 'देखिए', 32: 'ये', 33: 'क्या', 34: 'यह', 35: 'किया', 36: 'रहा', 37: 'रही', 38: 'रहे', 39: 'गया', 40: 'करने', 41: 'hallabol', 42: 'via', 43: 'anjanaomkashyap', 44: 'फिल्म', 45: 'बाद', 46: 'तक', 47: 'अब', 48: 'अपने', 49: 'कुछ', 50: 'दिया', 51: 'पुलिस', 52: 'था', 53: 'कहा', 54: 'लोगों', 55: 'कोई', 56: 'गई', 57: 'अपनी', 58: 'लेकिन', 59: 'वह', 60: 'बहुत', 61: 'foz5qpkk43', 62: 'जो', 63: 'मोदी', 64: 'मैं', 65: 'देश', 66: 'होने', 67: 'live', 68: 'आज', 69: 'हुए', 70: 'बजे', 71: '…', 72: 'exclusive', 73: 'बात', 74: 'वाले', 75: 'सरकार', 76: 'पहले', 77: 'टीम', 78: 'mobiletak', 79: 'भारत', 80: 'गए', 81: 'उनके', 82: 'जा', 83: 'न', 84: 'फिर', 85: 'हुआ', 86: 'तरह', 87: 'किसी', 88: 'tak

In [16]:
d = 300  # Embedding dimensionality


making embedding matrix contain vector corrus. each word in vocab

In [23]:


embedding_matrix_1 = np.zeros((len(word_index) + 1, d))
for index, word in index_word.items():
  if word in loaded_model:
    embedding_matrix_1[index,: ] = loaded_model[word]


KeyboardInterrupt: ignored

In [29]:
path="/content/drive/MyDrive/iisc/reverse_dictionary/embedding_matrix_1.npy"


In [18]:
#np.save(path, embedding_matrix_1)

In [30]:
embedding_matrix_1=np.load(path)

In [35]:

embedding_layer = Embedding(24550,
                            300,
                            weights=[embedding_matrix_1.T],
                            input_length=100,
                            trainable=False)

RNN model

In [20]:
# Build the RNN model
model_rnn = Sequential()
model_rnn.add(embedding_layer)  # Embedding layer
model_rnn.add(SimpleRNN(64,input_shape=(100,1),return_sequences=False))  # RNN encoder
model_rnn.add(Dense(64, activation='sigmoid'))  # Feedforward decoder
model_rnn.add(Dense(3, activation='softmax'))  # Output layer

# Compile the model
model_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model_rnn.summary()

# Train the model
model_rnn.fit(x_train, y_train, epochs=20, batch_size=64)

# Evaluate the model on test data
loss, accuracy = model_rnn.evaluate(x_val, y_val)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 300)          6460800   
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                23360     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 6,488,515
Trainable params: 27,715
Non-trainable params: 6,460,800
_________________________________________________________________
Epoch 1/20
114/114 [==============================] - 18s 96ms/step - loss: 1.0483 - accuracy: 0.4254
Epoch 2/20
114/114 [==============================] - 10s 85ms/step - l

LSTM model

In [36]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 100, 300)          7365000   
                                                                 
 dropout (Dropout)           (None, 100, 300)          0         
                                                                 
 lstm (LSTM)                 (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 3)                 303       
                                                                 
Total params: 7,525,703
Trainable params: 160,703
Non-trainable params: 7,365,000
_________________________________________________________________


In [37]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [38]:
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]

In [24]:
y_train.shape


(7262, 3)

In [39]:
history = model.fit(x_train, y_train,
                    batch_size=64,
                    epochs=32,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/32
103/103 [==============================] - ETA: 0s - loss: 1.0204 - accuracy: 0.4936

103/103 [==============================] - 55s 450ms/step - loss: 1.0204 - accuracy: 0.4936 - val_loss: 1.1210 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/32
103/103 [==============================] - ETA: 0s - loss: 1.0144 - accuracy: 0.4966

103/103 [==============================] - 46s 451ms/step - loss: 1.0144 - accuracy: 0.4966 - val_loss: 1.1003 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/32
103/103 [==============================] - ETA: 0s - loss: 1.0121 - accuracy: 0.4973

103/103 [==============================] - 45s 435ms/step - loss: 1.0121 - accuracy: 0.4973 - val_loss: 1.1474 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/32
103/103 [==============================] - ETA: 0s - loss: 1.0085 - accuracy: 0.4976

103/103 [==============================] - 48s 466ms/step - loss: 1.0085 - accuracy: 0.4976 - val_loss: 1.2086 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 5/32
103/103 [==============================] - ETA: 0s - loss: 1.0089 - accuracy: 0.4976

...

103/103 [==============================] - 48s 462ms/step - loss: 0.9775 - accuracy: 0.5145 - val_loss: 1.1204 - val_accuracy: 0.0839 - lr: 1.0000e-07
Epoch 29/32
103/103 [==============================] - ETA: 0s - loss: 0.9806 - accuracy: 0.5131

103/103 [==============================] - 47s 458ms/step - loss: 0.9806 - accuracy: 0.5131 - val_loss: 1.1204 - val_accuracy: 0.0839 - lr: 1.0000e-07
Epoch 30/32
103/103 [==============================] - ETA: 0s - loss: 0.9781 - accuracy: 0.5171

103/103 [==============================] - 45s 436ms/step - loss: 0.9781 - accuracy: 0.5171 - val_loss: 1.1204 - val_accuracy: 0.0839 - lr: 1.0000e-07
Epoch 31/32
103/103 [==============================] - ETA: 0s - loss: 0.9751 - accuracy: 0.5184

103/103 [==============================] - 45s 433ms/step - loss: 0.9751 - accuracy: 0.5184 - val_loss: 1.1204 - val_accuracy: 0.0839 - lr: 1.0000e-07
Epoch 32/32
103/103 [==============================] - ETA: 0s - loss: 0.9802 - accuracy: 0.5160

103/103 [==============================] - 45s 436ms/step - loss: 0.9802 - accuracy: 0.5160 - val_loss: 1.1204 - val_accuracy: 0.0839 - lr: 1.0000e-07


In [42]:
test_loss, test_accuracy = model.evaluate(x_val, y_val, verbose=1)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

57/57 [==============================] - 3s 44ms/step - loss: 1.7179 - accuracy: 0.0121
Test Loss: 1.717928171157837
Test Accuracy: 0.012121211737394333


In [41]:
y_label

array([[0.30277613, 0.15125005, 0.5459738 ],
       [0.4050103 , 0.12866725, 0.4663224 ],
       [0.30340734, 0.24288982, 0.45370284],
       ...,
       [0.27312076, 0.17525978, 0.5516194 ],
       [0.36001077, 0.16985786, 0.47013134],
       [0.385509  , 0.15696026, 0.4575307 ]], dtype=float32)

decision tree

In [42]:
from sklearn import tree

clf=tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(x_train,y_train)
clf.score(x_test,y_test)

0.5546989720998532

Nbow model

In [32]:
from collections import Counter
vocab_counter = Counter()
word_to_index = {}
index_to_word = {}
# Tokenization and vocabulary creation
for text in input:
    tokens = text.split()  # You can use more sophisticated tokenization methods if needed
    vocab_counter.update(tokens)
# Create a vocabulary list
vocab = list(vocab_counter.keys())
for index, word in enumerate(vocab):
    word_to_index[word] = index
    index_to_word[index] = word

# Print the vocabulary with indices
print("Vocabulary:", vocab)
print("Word to Index:", word_to_index)
print("Index to Word:", index_to_word)


Vocabulary: ['लोग', 'वतन', 'तक', 'खा', 'जाते', 'हैं', 'इसका', 'इसे', 'यकीन', 'नहींमान', 'जाएगा', 'तू', 'ले', 'जाकर', 'दिल्ली', 'दिखा', 'ला', 'दोस्त', 'गुमनाम', 'है', 'पर', 'मिटने', 'वाले', 'आतन्कवादियों', 'से', 'मिलकर', 'मशहूर', 'हो', 'गये', 'ज़ंजीर', 'बदली', 'जा', 'रही', 'थी', 'मैं', 'समझा', 'था', 'रिहाई', 'गयी', 'यूपी', 'में', 'बड़े', 'स्तर', 'दंगे', 'करवा', 'सकती', 'बीजेपी:', 'मुलायम', 'सिंह', 'यादव', '-', 'Navbharat', 'Times', 'अंग्रेजी', 'नहीं', 'आती', 'इसलिए', 'हिन्दी', 'ट्विट', 'ज्यादा', 'करता', 'हूँ', '।', 'निजी', 'राय', 'बुरा', 'लगे', 'तो', 'भाड़', 'मे', '|', 'कश्मीर', 'रहा', 'जल', 'जिहाद', 'परमपिता', 'परमेश्वर', 'अगर', 'हिंदुस्तान', 'मेँ', 'दंगा', 'होता', 'हे', 'ओर', 'मनुष्य', 'मरते', 'पुलिस', 'झुकना', 'माना', 'नहीँ', 'चाहिए', 'भारत', 'संघर्ष', 'आदमी', 'अकेला', 'सफलता', 'दुनिया', 'उसके', 'साथ', 'होती', 'जब', 'जग', 'उस', 'हसा', 'तब', 'उसी', 'ने', 'नया', 'इतिहास', 'रचा', 'है।', 'मोहब्बत', 'अंधी', 'तीखी', 'बात', 'राम', 'जेठमलानी', 'गड्गरी', 'इल्जाम', 'लगा', 'दिए', 'की', 'किसी', 

In [36]:
d = 300  # Embedding dimensionality
V = len(vocab)  # Vocabulary size

embedding_matrix = np.zeros((d, V))
for index, word in index_to_word.items():
  if word in loaded_model:
    embedding_matrix[:, index] = loaded_model[word]



In [37]:
path_new="/content/drive/MyDrive/iisc/reverse_dictionary/embedding_matrix.npy"
np.save(path, embedding_matrix)
embedding_matrix=np.load(path_new)

In [38]:
input_data=np.zeros((len(input),300))
for i in range(len(input)):
  tokens = input[i].split()
  vector=np.zeros(300)
  for word in tokens:
    index=word_to_index[word]
    vector =vector+embedding_matrix[:,index]
  input_data[i,:]=vector/len(tokens)

In [39]:
x_train,x_test,y_train,y_test=train_test_split(input_data,label,train_size=0.7,random_state=42)

In [40]:
print(pd.DataFrame(x_train),pd.DataFrame(y_train))

           0         1         2         3         4         5         6    \
0    -0.002578 -0.010392  0.008051  0.011773 -0.002116  0.029032 -0.002906   
1    -0.003904 -0.031838 -0.005357  0.001738 -0.014229  0.014997 -0.002685   
2    -0.003138 -0.020785  0.010745  0.003110  0.003689  0.014691  0.018565   
3     0.006374 -0.015570  0.007095  0.005639  0.005441 -0.014120 -0.019973   
4    -0.000224 -0.009408  0.028049  0.016233 -0.006753 -0.001152 -0.007206   
...        ...       ...       ...       ...       ...       ...       ...   
6348 -0.029130 -0.009860  0.032982 -0.002324 -0.018795 -0.004289  0.021149   
6349 -0.013921 -0.043576  0.013839 -0.016723  0.009417  0.026020 -0.037620   
6350  0.006761 -0.015625  0.042069  0.018656  0.005972 -0.006735  0.025191   
6351 -0.001444 -0.011462  0.029580 -0.002582  0.011900 -0.025128 -0.007485   
6352  0.011811 -0.039861  0.006539  0.001162 -0.027246 -0.002750  0.009849   

           7         8         9    ...       290       291    

In [41]:
# Replace NaN values with zeros
x_train[np.isnan(x_train)] = 0
x_test[np.isnan(x_test)] = 0
# Now you can proceed with your computations or analyses


DAN

In [43]:
x_train,x_test,y_train,y_test=train_test_split(input_data,onehot,train_size=0.7,random_state=42)

In [44]:
model_dan=Sequential()
model_dan.add(Dense(300,activation='linear',input_dim=300))
model_dan.add(Dense(100,activation='elu'))
model_dan.add(Dense(100,activation='elu'))
model_dan.add(Dense(100,activation='elu'))
model_dan.add(Dense(3,activation='softmax'))
model_dan.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_dan.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 300)               90300     
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 dense_5 (Dense)             (None, 100)               10100     
                                                                 
 dense_6 (Dense)             (None, 100)               10100     
                                                                 
 dense_7 (Dense)             (None, 3)                 303       
                                                                 
Total params: 140,903
Trainable params: 140,903
Non-trainable params: 0
_________________________________________________________________


In [45]:
print(len(x_train),len(y_train))

6353 6353


In [46]:
model_dan.fit(x_train, y_train, epochs=200, batch_size=64)

Epoch 1/200
100/100 [==============================] - 2s 4ms/step - loss: 0.8878 - accuracy: 0.5497
Epoch 2/200
100/100 [==============================] - 0s 4ms/step - loss: 0.7919 - accuracy: 0.6288
Epoch 3/200
100/100 [==============================] - 0s 4ms/step - loss: 0.7680 - accuracy: 0.6476
Epoch 4/200
100/100 [==============================] - 0s 4ms/step - loss: 0.7569 - accuracy: 0.6474
Epoch 5/200
100/100 [==============================] - 0s 4ms/step - loss: 0.7536 - accuracy: 0.6575
Epoch 6/200
100/100 [==============================] - 0s 4ms/step - loss: 0.7449 - accuracy: 0.6592
Epoch 7/200
100/100 [==============================] - 0s 4ms/step - loss: 0.7383 - accuracy: 0.6683
Epoch 8/200
100/100 [==============================] - 0s 4ms/step - loss: 0.7421 - accuracy: 0.6649
Epoch 9/200
100/100 [==============================] - 0s 4ms/step - loss: 0.7353 - accuracy: 0.6665
Epoch 10/200
100/100 [==============================] - 0s 4ms/step - loss: 0.7305 - accura

In [47]:
loss, accuracy = model_dan.evaluate(x_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

86/86 [==============================] - 0s 2ms/step - loss: 4.5289 - accuracy: 0.6454
Test loss: 4.5289, Test accuracy: 0.6454


bert

In [51]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(newdf, random_state=91)


In [49]:
! pip install simpletransformers

In [ ]:
bert-base-multilingual-uncased

In [50]:
from simpletransformers.classification import ClassificationModel

In [14]:
# set use_cuda=False on CPU-only platforms
bert = ClassificationModel('bert', 'bert-base-multilingual-uncased', num_labels=3, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 5,
    'silent': True
})


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
bert.train_model(train)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


(4255, 0.5142761288462459)

In [16]:
result, model_outputs, wrong_predictions = bert.eval_model(test)
bads = {}
for pred in wrong_predictions:
    if pred.label in bads:
        bads[pred.label] += 1
    else:
        bads[pred.label] = 1
print("wrong predictions")
print(str(len(wrong_predictions)) + ' wrong out of ' + str(len(test)))
bads

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


wrong predictions
493 wrong out of 2270


{2: 248, 0: 204, 1: 41}

In [59]:
print("accuracy",(2270-493)/2270)


accuracy 0.7828193832599118


electra

In [ ]:
model_electra = ClassificationModel('bert', 'monsoon-nlp/hindi-tpu-electra', num_labels=3, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 4,
})


In [54]:
model_electra.train_model(train)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/6807 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/851 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/851 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/851 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/851 [00:00<?, ?it/s]

(3404, 0.6402083646799367)

In [56]:
result, model_outputs, wrong_predictions = model_electra.eval_model(test)
bads = {}
for pred in wrong_predictions:
    if pred.label in bads:
        bads[pred.label] += 1
    else:
        bads[pred.label] = 1
print("wrong predictions")
print(str(len(wrong_predictions)) + ' wrong out of ' + str(len(test)))
bads

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2270 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/284 [00:00<?, ?it/s]

wrong predictions
528 wrong out of 2270


{0: 199, 2: 263, 1: 66}

In [60]:
print("accuracy",(2270-528)/2270)

accuracy 0.7674008810572687


In [44]:
from tabulate import tabulate
from PIL import Image, ImageDraw, ImageFont

# Define the data for the table
data = [
    ["DAN", "64.54%"],
    ["RNN", "67.65%"],
    ["LSTM", "51.60%"],
    ["Decision tree", "55.46%"]
]

# Define the headers for the table
headers = ["Model", "Accuracy"]

# Generate the formatted table
table = tabulate(data, headers, tablefmt="pretty")

# Create an image with the table using Pillow
font = ImageFont.load_default()  # You can customize the font if needed
img = Image.new('RGB', (800, 300), color='white')
d = ImageDraw.Draw(img)
d.text((10, 10), table, fill=(0, 0, 0), font=font)

# Save the image
img.save('accuracy_table_classi.png')


In [45]:
from tabulate import tabulate
from PIL import Image, ImageDraw, ImageFont

# Data
data = [
    ["BERT-base-multilingual-uncased", "Test Accuracy: 78.28%"],
    ["Indic-bert by AI Bharat (768 dimensions)", "Test Accuracy: 77.77%"],
    ["monsoon-nlp/hindi-tpu-electra", "Test Accuracy: 76.37%"],
    ["monsoon-nlp/hindi-bert (256 dimensions)", "Test Accuracy: 67.84%"],
    ["FastText (300 dimensions)", "Test Accuracy: 64.54%"]
]

# Convert data to tabulated format
table = tabulate(data, headers=["Embedding", "Accuracy"], tablefmt="plain")

# Create an image with the table
image = Image.new("RGB", (800, 200), color="white")
draw = ImageDraw.Draw(image)
font = ImageFont.load_default()

# Draw the table text
draw.text((10, 10), table, font=font, fill="black")

# Save the image
image.save("accuracy_table_embedding.png")